# VAE and AE

## 1. Load data

In [46]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

In [55]:
eye_data = np.load('eye/data.npy') # (1476, 50, 50, 3)
eye_label = np.load('eye/label.npy') # (1476, 1)

In [49]:
class EyeDataset(Dataset):
    def __init__(self, data, label, transform=None):
        self.data = data
        self.label = label
        self.transform = transform
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        data = self.data[idx]
        label = self.label[idx]
        if self.transform:
            data = self.transform(data)
        
        return (data, label)

In [54]:
eye_dataset = EyeDataset(data=eye_data, label=eye_label)

batch_size = 18  
eye_dataloader = DataLoader(eye_dataset, batch_size=batch_size, shuffle=False)
# for batch in eye_dataloader:
#     print(type(batch))
#     print(batch[0].shape)
#     print(batch)
#     break

## 2. Model construct

## 3. Train model